Objectifs :
- Implémenter le bag of word ;
- Implémenter le TF-IDF.

## Importations des packages

In [1]:
from nltk.tokenize import word_tokenize
from nltk.text import Text
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer
import nltk

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

## Importations des données

Les données sont issue de [cette base de données](https://cs.nyu.edu/~kcho/DMQA/).

Maintenant que nos outils sont chargés, nous allons charger nos données.

Cliquez sur le lien ci-dessous :

https://drive.google.com/drive/folders/12OmusfAUOcoLOCwEc--nfkKQ5eEozU45?usp=sharing

Cliquer droit sur le dossier data et appuyer sur ajouter à mon drive.

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


Les données sont maintenant dans votre environnement collab.

In [3]:
import os
print(os.listdir('gdrive/MyDrive/Exercice_1/Partie_1')[:10])

['0a3ff2f0a147c158845afa44d2a012064896566b.story', '0a3fff5779a8f7cfdde5d284a429ab89fd5e85df.story', '0a0f56ebc5a0a67ed18de79d99b40a42d8058d04.story', '0a3ad75d92c5bc2eccf2763df86afe5ddeffed75.story', '0a3f2400ba4e5cdf4b3638ae6fb60fdfa12a2680.story', '0a3f567efff9f0748b2758c9e8c17dc66beade04.story', '0a05b14962b2e73bbff82086762e0e23d32b359f.story', '0a1ad82d161d90d758240407cb8c8fcebff4a212.story', '0a4ec4d37683347ca62b53982d2c5f4efb86f444.story', '0a4b2d4ea5fb0625e3e747525062f0a85345e4df.story']


# Création de la base de données

In [5]:
cmpt = 0
dict_data = dict()

for file_name in os.listdir('gdrive/MyDrive/Exercice_1/Partie_1'):
    f = open('gdrive/MyDrive/Exercice_1/Partie_1/'+file_name, 'r')
    lst = ""
    for line in f:

       line.strip()
       line = line.replace("\n" ,'')
       line = line.replace("//" , '')
       line = line.replace("/" , '')
       if len(line) > 0 :
        lst += line

    dict_data[cmpt] = lst
    cmpt += 1
print(dict_data)

{0: 'LOS ANGELES, California (CNN)  -- After about 30 hours of deliberation, a jury on Monday convicted music producer Phil Spector of second-degree murder in the death of actress Lana Clarkson more than six years ago.Phil Spector will not remain free on bail pending his May 29 sentencing.Wearing a black suit with a red tie and pocket square, Spector showed no reaction as the verdict was announced. Now 69, he faces a sentence of 18 years to life in prison when he is sentenced May 29.Asked if he agreed to the sentencing date, Spector quietly answered, "Yes."Prosecutor Alan Jackson said he believed the jury reached the correct verdict, and he acknowledged the strength and backing of Clarkson\'s family. But defense attorney Doron Weinberg said Spector\'s defense team disagreed and planned to appeal."We don\'t believe justice was done," Weinberg said.Superior Court Judge Larry Paul Fidler declined to allow Spector to remain free on bail pending sentencing, citing Spector\'s years-long "pat

# Prétraitements

## En minuscule

In [8]:
dict_data_min = dict()
for k, v in dict_data.items():
  dict_data_min[k] = v.lower()
print(dict_data_min)

{0: 'los angeles, california (cnn)  -- after about 30 hours of deliberation, a jury on monday convicted music producer phil spector of second-degree murder in the death of actress lana clarkson more than six years ago.phil spector will not remain free on bail pending his may 29 sentencing.wearing a black suit with a red tie and pocket square, spector showed no reaction as the verdict was announced. now 69, he faces a sentence of 18 years to life in prison when he is sentenced may 29.asked if he agreed to the sentencing date, spector quietly answered, "yes."prosecutor alan jackson said he believed the jury reached the correct verdict, and he acknowledged the strength and backing of clarkson\'s family. but defense attorney doron weinberg said spector\'s defense team disagreed and planned to appeal."we don\'t believe justice was done," weinberg said.superior court judge larry paul fidler declined to allow spector to remain free on bail pending sentencing, citing spector\'s years-long "pat

## Tokenization

In [9]:
dict_data_token = dict()
for k, v in dict_data_min.items():
  tokenizer = nltk.RegexpTokenizer(r'\w+')
  tokens = tokenizer.tokenize(v)
  dict_data_token[k] = tokens
print(dict_data_token)

{0: ['los', 'angeles', 'california', 'cnn', 'after', 'about', '30', 'hours', 'of', 'deliberation', 'a', 'jury', 'on', 'monday', 'convicted', 'music', 'producer', 'phil', 'spector', 'of', 'second', 'degree', 'murder', 'in', 'the', 'death', 'of', 'actress', 'lana', 'clarkson', 'more', 'than', 'six', 'years', 'ago', 'phil', 'spector', 'will', 'not', 'remain', 'free', 'on', 'bail', 'pending', 'his', 'may', '29', 'sentencing', 'wearing', 'a', 'black', 'suit', 'with', 'a', 'red', 'tie', 'and', 'pocket', 'square', 'spector', 'showed', 'no', 'reaction', 'as', 'the', 'verdict', 'was', 'announced', 'now', '69', 'he', 'faces', 'a', 'sentence', 'of', '18', 'years', 'to', 'life', 'in', 'prison', 'when', 'he', 'is', 'sentenced', 'may', '29', 'asked', 'if', 'he', 'agreed', 'to', 'the', 'sentencing', 'date', 'spector', 'quietly', 'answered', 'yes', 'prosecutor', 'alan', 'jackson', 'said', 'he', 'believed', 'the', 'jury', 'reached', 'the', 'correct', 'verdict', 'and', 'he', 'acknowledged', 'the', 'stre

## Stopwords

In [10]:
dict_data_stop = dict()
for k, v in dict_data_token.items():
  dict_data_stop[k] = [w for w in v if not w in list(nltk.corpus.stopwords.words())]
print(dict_data_stop)

{0: ['angeles', 'california', 'cnn', '30', 'hours', 'deliberation', 'jury', 'monday', 'convicted', 'music', 'producer', 'phil', 'spector', 'degree', 'murder', 'death', 'actress', 'lana', 'clarkson', 'years', 'ago', 'phil', 'spector', 'remain', 'free', 'bail', 'pending', '29', 'sentencing', 'wearing', 'black', 'suit', 'red', 'tie', 'pocket', 'square', 'spector', 'showed', 'reaction', 'verdict', 'announced', '69', 'faces', 'sentence', '18', 'years', 'life', 'prison', 'sentenced', '29', 'asked', 'agreed', 'sentencing', 'date', 'spector', 'quietly', 'answered', 'prosecutor', 'alan', 'jackson', 'believed', 'jury', 'reached', 'correct', 'verdict', 'acknowledged', 'strength', 'backing', 'clarkson', 'family', 'defense', 'attorney', 'doron', 'weinberg', 'spector', 'defense', 'team', 'disagreed', 'planned', 'appeal', 'believe', 'justice', 'weinberg', 'superior', 'court', 'judge', 'larry', 'paul', 'fidler', 'declined', 'spector', 'remain', 'free', 'bail', 'pending', 'sentencing', 'citing', 'spect

## Stemming

In [12]:
dict_data_stem = dict()
st = LancasterStemmer()

for index, doc in dict_data_stop.items():
  dict_data_stem[index] = [st.stem(w) for w in doc ]
print(dict_data_stem)

{0: ['angel', 'californ', 'cnn', '30', 'hour', 'delib', 'jury', 'monday', 'convict', 'mus', 'produc', 'phil', 'spect', 'degr', 'murd', 'dea', 'actress', 'lan', 'clarkson', 'year', 'ago', 'phil', 'spect', 'remain', 'fre', 'bail', 'pend', '29', 'sent', 'wear', 'black', 'suit', 'red', 'tie', 'pocket', 'squ', 'spect', 'show', 'react', 'verdict', 'annount', '69', 'fac', 'sent', '18', 'year', 'lif', 'prison', 'sent', '29', 'ask', 'agree', 'sent', 'dat', 'spect', 'quiet', 'answ', 'prosecut', 'al', 'jackson', 'believ', 'jury', 'reach', 'correct', 'verdict', 'acknowledg', 'strength', 'back', 'clarkson', 'famy', 'defens', 'attorney', 'doron', 'weinberg', 'spect', 'defens', 'team', 'disagree', 'plan', 'ap', 'believ', 'just', 'weinberg', 'supery', 'court', 'judg', 'larry', 'paul', 'fidl', 'declin', 'spect', 'remain', 'fre', 'bail', 'pend', 'sent', 'cit', 'spect', 'year', 'long', 'pattern', 'viol', 'involv', 'firearm', 'watch', 'lawy', 'giv', 'view', 'verdict', 'isol', 'incid', 'fidl', 'not', 'spec

# Bag of words

## Vecteur de vocabulaire

In [13]:
vocab = set()

for index, doc in dict_data_stem.items():
  for word in doc :
    vocab.add(word)

print(len(vocab))

4016


## Bag of words

In [18]:
# Convert vocab from a set to a list
vocab_list = list(vocab)

data = pd.DataFrame([], columns=vocab_list)

for index, doc in dict_data_stem.items() :
  # Your Code


0 ['angel', 'californ', 'cnn', '30', 'hour', 'delib', 'jury', 'monday', 'convict', 'mus', 'produc', 'phil', 'spect', 'degr', 'murd', 'dea', 'actress', 'lan', 'clarkson', 'year', 'ago', 'phil', 'spect', 'remain', 'fre', 'bail', 'pend', '29', 'sent', 'wear', 'black', 'suit', 'red', 'tie', 'pocket', 'squ', 'spect', 'show', 'react', 'verdict', 'annount', '69', 'fac', 'sent', '18', 'year', 'lif', 'prison', 'sent', '29', 'ask', 'agree', 'sent', 'dat', 'spect', 'quiet', 'answ', 'prosecut', 'al', 'jackson', 'believ', 'jury', 'reach', 'correct', 'verdict', 'acknowledg', 'strength', 'back', 'clarkson', 'famy', 'defens', 'attorney', 'doron', 'weinberg', 'spect', 'defens', 'team', 'disagree', 'plan', 'ap', 'believ', 'just', 'weinberg', 'supery', 'court', 'judg', 'larry', 'paul', 'fidl', 'declin', 'spect', 'remain', 'fre', 'bail', 'pend', 'sent', 'cit', 'spect', 'year', 'long', 'pattern', 'viol', 'involv', 'firearm', 'watch', 'lawy', 'giv', 'view', 'verdict', 'isol', 'incid', 'fidl', 'not', 'spect'